# 按照自由時報那篇格式處理
[助教的程式碼](https://github.com/MiccWan/Political-News-Analysis/blob/master/crawler/new_talk_crawler.ipynb)

In [5]:
from selenium import webdriver
import numpy as np
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from pathlib import Path, PureWindowsPath
from selenium.webdriver.chrome.options import Options

import sys
import pickle
import requests
from datetime import datetime



# 參考用的
def process_document(document, date):
    
   
    nodes = document.select('ul.list > li')
    #nodes = document.find_all(class_="_2bF theme-list")
    
    data = list()

    for li in nodes:
        # check if is empty element
        if li.select_one('a') == None:
            continue

        # get link
        li_link = 'http://news.ltn.com.tw/' + li.select_one('a')['href']
        #li_link = 'https://news.cnyes.com' + li.select_one('a')['href']
        

        # request for document
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')

        # get date
        li_date = datetime.strptime(date, "%Y%m%d").strftime('%Y-%m-%d')

        #get title
        li_title = li.select_one('p').get_text()

        #get content
        li_content = ""
        for ele in li_doc.select('div.text > p'):
            if not 'appE1121' in ele.get('class', []):
                li_content += ele.get_text()

        # append new row
        data.append({
            'date' : li_date,
            'title': li_title,
            'link' : li_link,
            'content' : li_content,
            'tags' : []
        })
    return data

In [7]:
res = requests.get('https://news.cnyes.com/news/cat/tw_stock_news')
doc = bs(res.text, 'lxml')


In [68]:
#nodes = doc.find_all(class_="_2bF theme-list")
nodes = doc.find_all(class_="_1Zd")
nodes[0].get('href')

'/news/id/4298273'

In [93]:
data = list()
i = 0
for li in nodes:
        # get link
        li_link = 'https://news.cnyes.com' + li.get('href')
        
        # request for document 由各個網站中在抓取該新聞的資料
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')
        
        li_content = ""
        for ele in li_doc.find_all('p'):
            if not 'appE1121' in ele.get('class', []):
                li_content += ele.get_text()
        
        data.append({
            'link' : li_link,
            'content' : li_content
        })
        print("第%d則好了"%(i),end = '')
        i = i+1
data

第0則好了第1則好了第2則好了第3則好了第4則好了第5則好了第6則好了第7則好了第8則好了第9則好了第10則好了第11則好了第12則好了第13則好了第14則好了第15則好了第16則好了第17則好了第18則好了第19則好了第20則好了第21則好了第22則好了第23則好了第24則好了第25則好了第26則好了第27則好了第28則好了第29則好了

[{'link': 'https://news.cnyes.com/news/id/4298273',
  'content': '華為拉貨升溫，除了向台積電 (2330-TW) 擴大追單外，對功率放大器 (PA)、散熱需求同步看俏，相關廠商如砷化鎵的穩懋 (3105-TW)、全新 (2455-TW)，散熱模組的雙鴻 (3224-TW)、奇鋐 (3017-TW) 第 2 季營運添動能。華為在今年推出新款旗艦機 P30 系列，加上在美中貿易戰下，更上修全年手機出貨量，由 2.5-2.6 億支上修至 2.8 億支，與去年出貨量 2 億支相比，年增約 33.6%。華為也向台積電追加 7 奈米大單，將安全庫存由 2 個月拉升到 6 個月，除了帶動晶片需求成長外，市場也看好對 PA、散熱模組的需求也將同步升溫。散熱模組業者雙鴻日前就在法說會上指出，新款手機採用新晶片，對散熱需求增加，雖然第 2 季傳統淡季，但在需求支撐下，熱板或是熱管的跌價壓力並沒有那麼大。奇鋐也表示，華為持續擴大在手機、基地台等通訊應用布局，可望帶動散熱訂單需求大幅成長，據了解，華為是奇鋐散熱產品的第 2 大客戶。此外，華為手機出貨攀升，對 PA 需求同步看俏，市場看好砷化鎵族群中的穩懋、全新第 2 季營收可望重返正成長，加上下半年迎來傳統旺季，伴隨 5G 需求顯現，營運可望逐季升溫。穩懋第 1 季營收 36.1 億元，季減 14.3%、年減 19.1%，優於公司當初預估的季減 17-19%；全新第 1 季營收 4.71 億元，季減 3.7%、年減 16.3%。'},
 {'link': 'https://news.cnyes.com/news/id/4298156',
  'content': '第 1 季在美中貿易大戰影響下，客戶端積極調降庫存影響，臻鼎 - KY(4958-TW)、華通 (2313-TW) 等大型 PCB 廠營收陷入谷底，但在於貿易戰趨緩，業者陸續感受到目前市場恐慌情緒，已較去年 12 月明顯好轉，且在客戶端庫存偏低的情況下，有助第 2 季市場需求提升。同時，全球\xa05G 通訊競爭態勢逐漸明朗，5G(Pre-5G) 需求逐步浮現，健鼎主管就指指出，第 2 季起， 5G 的布局需求，有助提升 PCB 產業的訂單能量。臻鼎董事長沈慶芳表示，預估 5G 全面進入商業化應用階段，時間點應在

In [ ]:
<a class="_1Zd" title="〈Q2產業景氣展望〉華為手機出貨旺 散熱、砷化鎵族群Q2營運添動能" href="/news/id/4298273" style="display: block;"><div class="_67t theme-meta"><time datetime="2019-04-07T16:30:06+08:00">16:30</time><div class="_jFb theme-sub-cat">台股新聞</div></div><div class="_1xc"><h3>〈Q2產業景氣展望〉華為手機出貨旺 散熱、砷化鎵族群Q2營運添動能</h3><figure class=""><img src="https://cimg.cnyes.cool/prod/news/4298273/xs/b6168fe977414bb74f79345108bae357.jpg" srcset="https://cimg.cnyes.cool/prod/news/4298273/xs/b6168fe977414bb74f79345108bae357.jpg 1x,https://cimg.cnyes.cool/prod/news/4298273/s/b6168fe977414bb74f79345108bae357.jpg 2x,https://cimg.cnyes.cool/prod/news/4298273/m/b6168fe977414bb74f79345108bae357.jpg 3x" alt=""><!-- react-text: 2587 --><!-- /react-text --></figure></div></a>